In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
df_player = pd.read_csv('../../../Data/Stats_csv/players.csv')

In [4]:
lst_name_defender = df_player.loc[df_player['position']=='Defender', ['name','link_profile_stats_2425']].values

In [ ]:
def getPathHTMLByPosition(position, section):
    lst_name_defender = df_player.loc[df_player['position']==position, ['name','link_profile_stats_2425']].values
    pathHtmls = []
    for name, link_profile_stats_2425 in lst_name_defender:
        pathHtmls.append([name, link_profile_stats_2425 , f'../../../Data/players/{name} - stats{section}.html' ])
    return pathHtmls

In [6]:
import nbimporter
import crawl_players_stats

In [14]:
def checkCrawlByName(pathHTML):
    name, link_profile_stats_2425, path = pathHTML
    retry_limit = 3  
    success = 0 
    retry_count = 0 
    
    while success < 2 and retry_count < retry_limit:
        try:
            with open(path, "r", encoding="utf-8") as file:
                content = file.read()
            soup = BeautifulSoup(content, "html.parser")
            
            name_div = soup.find("div", class_="player-header__name t-colour")
            
            if name_div:
                first_name = name_div.find("div", class_="player-header__name-first")
                first_name = first_name.text.strip() if first_name else ""  # Xử lý first_name
                last_name = name_div.find("div", class_="player-header__name-last").text.strip()
                full_name = f"{first_name} {last_name}"
                print(full_name)
                success = 2  # Dữ liệu thành công, thoát vòng lặp
            else:
                print(f"Không tìm thấy thẻ với class 'player-header__name t-colour' cho {name}")
                print(f"Thực hiện lại crawl lần {retry_count + 1}")
                crawl_players_stats.get_HTML_Info([link_profile_stats_2425, path])
                retry_count += 1  # Increment retry counter
        except Exception as e:
            print(f"Lỗi trong quá trình xử lý {name}: {e}")
            print(f"Thực hiện lại crawl lần {retry_count + 1}")
            crawl_players_stats.get_HTML_Info([link_profile_stats_2425, path])
            retry_count += 1  # Increment retry counter

    if retry_count >= retry_limit:
        print(f"Đã thử {retry_limit} lần mà không thành công.")


In [ ]:
def getPlayerDetails(html_source):
    dict = {}

    soup = BeautifulSoup(html_source , 'html.parser')

    try:
        dict['Name'] = soup.find(class_='player-header__name-first').text.strip() + ' ' + soup.find(class_='player-header__name-last').text.strip()
    except (AttributeError,KeyError):
        print("Skipping incomplete stat detail.")

    try:
        dict['Wins'] = soup.find(class_='allStatContainer js-all-stat-container statwins').text.strip()
    except (AttributeError,KeyError):
        print("Skipping incomplete stat detail.")

    try:
        dict['Losses'] = soup.find(class_='allStatContainer js-all-stat-container statlosses').text.strip()
    except (AttributeError,KeyError):
        print("Skipping incomplete stat detail.")

    #Thông tin cá nhân
    personalDetails = soup.find_all("div", class_='player-overview__col')
    for personalDetail in personalDetails:
        title_element = personalDetail.find(class_='player-overview__label')
        value_element = personalDetail.find(class_='player-overview__info')
        
        # Check if both the title and value elements are found
        if title_element and value_element:
            title = title_element.text.strip()  # Get the stat name
            value = value_element.text.strip()  # Get the stat value
            dict[title] = value
        else:
            print("Skipping incomplete stat detail.")

    #Do Appearances có ở thông tin cá nhân là của tất mùa giải, ghi đè Appearances ở đây là của chỉ mùa giải hiện tại
    try:
        dict['Appearances'] = soup.find(class_='allStatContainer js-all-stat-container statappearances').text.strip()
    except (AttributeError,KeyError):
        print("Skipping incomplete stat detail.")

    try: 
        dict['Nationality'] = soup.find(class_='player-overview__player-country').text.strip()
    except (AttributeError,KeyError):
        print("Skipping incomplete stat detail.")

    return dict

In [12]:
def checkAndReCrawl():
    pathHTMLs = getPathHTMLByPosition('Defender', '2425')
    pathHTMLs += getPathHTMLByPosition('Forward', '2425')
    pathHTMLs += getPathHTMLByPosition('Goalkeeper', '2425')
    pathHTMLs += getPathHTMLByPosition('Midfielder', '2425')
    for pathHTML in pathHTMLs:
        checkCrawlByName(pathHTML)



In [15]:
checkAndReCrawl()

Max Aarons
Zach Abbott
Josh Acheampong
Tosin Adarabioyo
Tayo Adaramola
Valintino Adedokun
Nayef Aguerd
Ola Aina
Rayan Aït-Nouri
Kristoffer Ajer
Manuel Akanji
Nathan Aké
Trent Alexander-Arnold
Sonny Aljofree
Max Alleyne
Harry Amass
Joachim Andersen
Aaron Anselmino
Julián Araujo
Benjamin Arthur
Harrison Ashby
Noël Atom
Sèrge Aurier
Benoît Badiashile
Elkan Baggott
Eric Bailly
Dominic Ball
Fodé Ballo-Touré
Valentín Barco
Calvin Bassey
Mads Bech
Owen Beck
Jan Bednarek
Armel Bella-Kotchap
Ben Nelson
Rhys Bennett
Giulian Biancone
Bendegúz Bolla
Willy-Arnaud Boly
Sven Botman
Conor Bradley
Zak Bradshaw
Kaden Braithwaite
Jarrad Branthwaite
James Bree
Josh Brooking
Hugo Bueno
Cameron Burgess
Dan Burn
Luke Butterfield
Riccardo Calafiori
Duje Caleta-Car
Elijah Campbell
Álvaro Carreras
Kaelan Casey
Matty Cash
Timothy Castagne
Cédric Soares
Trevoh Chalobah
Calum Chambers
Dylan Charlton
Ben Chilwell
Nathaniel Clyne
Conor Coady
Séamus Coleman
Nathan Collins
Levi Colwill
Vladimír Coufal
Tristan Crama
Aa